<h1 align=center><font size = 5>Want to open a New Multiplex in Tampa? Here's what data says</font></h1>

## Introduction
Aim of this project is to analyze and explore the current theatres locations and neighbourhoods in Tampa and recommend best place to Open a New Theatre in Tampa, Florida, United States .

Steps Involved 

1. Build a dataframe of neighborhoods in Tampa, Florida by web scraping the data from Wikipedia <a href="(https://en.wikipedia.org/wiki/Neighborhoods_in_Tampa,_Florida)">page</a> of Tampa Neighbourhoods using BeautifulSoup library.
2. Obtaing geographical coordinates of each neighborhood using Geopy library. 
3. Get current location of movie theatres in and around each neighbourhood using data from Foursquare API
4. Explore and Analyse the neighbourhoods from the data obtained and cluster the localities use folium to visualise neighbourhoods and clusters.
4. Select the best cluster for opening multiplex.


Import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests

print('Libraries imported.')


Libraries imported.


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Webscraping and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

## 1. Webscraping and Explore Dataset

This page has List of Neigbhourhoods in Tampa. In order to segement the neighborhoods and explore them, we will essentially need a dataset of neighbourhoods as well as the the latitude and logitude coordinates of each neighborhood. 

In the below cell, lets use the BeautifulSoup library imported to do webscrapping and requests library to get text data from the webpage.
Lets get the code from the website HTML page.

In [2]:
Data=requests.get("https://en.wikipedia.org/wiki/Neighborhoods_in_Tampa,_Florida").text
soup = BeautifulSoup(Data,'lxml')
#soup
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Neighborhoods in Tampa, Florida - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh8YNwpAIEIAADYSafMAAAAT","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Neighborhoods_in_Tampa,_Florida","wgTitle":"Neighborhoods in Tampa, Florida","wgCurRevisionId":880697949,"wgRevisionId":880697949,"wgArticleId":2622802,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive t

In [3]:
# create a list to store neighborhood data and get list of neighborhoods in Tampa
neighborhoodList = []
for row in soup.find('div', class_='mw-parser-output').find_all('table')[1].find_all('td')[0].find('ul').find_all('li'):
    neighborhoodList.append(row.text)
for row in soup.find('div', class_='mw-parser-output').find_all('table')[1].find_all('td')[1].find('ul').find_all('li'):
    neighborhoodList.append(row.text)
for row in soup.find('div', class_='mw-parser-output').find_all('table')[1].find_all('td')[2].find('ul').find_all('li'):
    neighborhoodList.append(row.text)  
neighborhoodList

['Armenia Gardens Estates',
 'Audubon Park',
 'Ballast Point',
 'Bayshore Beautiful',
 'Bayshore Gardens',
 'Bayside West',
 'Beach Park',
 'Beach Park Isles',
 'Beasley',
 'Bel Mar Shores',
 'Bon Air',
 'Carver City-Lincoln Gardens',
 'Channel District',
 'College Hill',
 'Culbreath Bayou',
 'Culbreath Heights',
 'Culbreath Isles',
 'Davis Islands',
 'Dixie Farms',
 'Drew Park',
 'Eastern Heights',
 'East Tampa\nBelmont Heights\nJackson Heights\nRainbow Heights',
 'Belmont Heights',
 'Jackson Heights',
 'Rainbow Heights',
 'East Ybor',
 'Fair Oaks - Manhattan Manor',
 'Fern Cliff',
 'Florence Villa',
 'Forest Hills',
 'Gandy-Sun Bay South',
 'Golf View',
 'Grant Park',
 'Gray Gables',
 'Harvey Heights',
 'Highland Pines',
 'Historic Hyde Park North',
 'Hyde Park Spanishtown Creek',
 'Harbour Island',
 "Hunter's Green",
 'Interbay',
 'Live Oaks Square',
 'Lowry Park',
 'Lowry Park North',
 'Maryland Manor',
 'New Suburb Beautiful',
 'North Bon Air',
 'Northeast Community',
 'North Hyde

In [1]:
#converting list into a pandas dataframe of neighborhoods
Tpo_df = pd.DataFrame({"Neighborhood": neighborhoodList})
Tpo_df.head(10)

NameError: name 'pd' is not defined

In [8]:
import pandas as pd
Tpo_df=pd.DataFrame(Tpo_df,columns=['Neighborhood'])
Tpo_df.head(10)

,Neighborhood
0,Armenia Gardens Estates
1,Audubon Park
2,Ballast Point
3,Bayshore Beautiful
4,Bayshore Gardens
5,Bayside West
6,Beach Park
7,Beach Park Isles
8,Beasley
9,Bel Mar Shores


Now that we have all neighbourhoods in the list lets get the coordinates of each neighbourhood using Geopy Library

In [9]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Tampa, Florida'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords


Getting coordinates for all the neighborhoods in the dataframe

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Tpo_df["Neighborhood"].tolist() ]
coords

[[28.018050000000073, -82.48137999999994],
 [28.077426974307436, -82.41697426552727],
 [27.88062000000008, -82.48840999999999],
 [27.901890000000037, -82.49604999999997],
 [27.92515000000003, -82.48695999999995],
 [27.935967966714355, -82.52692641477772],
 [27.93948000000006, -82.52323999999999],
 [27.933620000000076, -82.53087999999997],
 [28.020627487045026, -82.5674911883877],
 [27.911670000000072, -82.52893999999998],
 [28.011738291554018, -82.39228053131151],
 [27.95741000000004, -82.51382999999998],
 [27.948710000000062, -82.44618999999994],
 [27.975970000000075, -82.43168999999995],
 [27.93231000000003, -82.52146999999997],
 [27.93074000000007, -82.51614999999998],
 [27.931740000000048, -82.52856999999995],
 [27.922390000000064, -82.45606999999995],
 [27.96180000000004, -82.39299999999997],
 [27.986500000000035, -82.51327999999995],
 [27.98618608269957, -82.4104821683383],
 [27.981390000000033, -82.42799999999994],
 [28.099171253380234, -82.40306797339781],
 [28.05483600000001, 

Creating dataframe by adding coordinates of each neighborhood 

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
#merge to main dataset
Tpo_df['Latitude'] = df_coords['Latitude']
Tpo_df['Longitude'] = df_coords['Longitude']
Tpo_df

,Neighborhood,Latitude,Longitude
0,Armenia Gardens Estates,28.018050,-82.481380
1,Audubon Park,28.077427,-82.416974
2,Ballast Point,27.880620,-82.488410
3,Bayshore Beautiful,27.901890,-82.496050
4,Bayshore Gardens,27.925150,-82.486960
5,Bayside West,27.935968,-82.526926
6,Beach Park,27.939480,-82.523240
7,Beach Park Isles,27.933620,-82.530880
8,Beasley,28.020627,-82.567491
9,Bel Mar Shores,27.911670,-82.528940


save Geodata into csv in the local machine

In [13]:
# save the DataFrame as CSV file
Tpo_df.to_csv("Tpo_df.csv", index=False)

Visualising the neighborhood in Tamap

In [14]:
# create map of Tampa using latitude and longitude values
map_Tampa = folium.Map(location=[27.94752,-82.45843], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Tpo_df['Latitude'], Tpo_df['Longitude'], Tpo_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Tampa)  
    
map_Tampa

#### Define Foursquare API Credentials and Version 

In [15]:
CLIENT_ID = 'BMD2ISRPVJGGDDNKNEOA0ZTERCO3YJNPSUQEVVFE53B4CEVR' # your Foursquare ID
CLIENT_SECRET = '0WVRSM3UKH3K0LJLS1GNCUVTNCVGBG4AAWPZWI3TD2YHV5OF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BMD2ISRPVJGGDDNKNEOA0ZTERCO3YJNPSUQEVVFE53B4CEVR
CLIENT_SECRET:0WVRSM3UKH3K0LJLS1GNCUVTNCVGBG4AAWPZWI3TD2YHV5OF


Since Specific UPI premium calls needs premium subscription, I used method of getting top 100 venues in the vicinity of neighborhood and  filtering the category of movie theatres. Getting 100 venues within the 5000m range of neighborhood from Foursquare API and storing results in the dataframe.

In [16]:
radius = 5000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Tpo_df['Latitude'], Tpo_df['Longitude'], Tpo_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [17]:
#results = requests.get(url).json()
#results

In [18]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
#checking if we got the venues of movie theater
x=venues_df[venues_df['VenueCategory']=='Movie Theater']
x['VenueName'].unique()


(9510, 7)


array(['Studio Movie Grill Tampa', 'CinéBistro at Hyde Park Village',
       'AMC West Shore 14', 'AMC Veterans 24', 'Tampa Theatre',
       'AMC Highwoods 20', 'AMC The Regency 20'], dtype=object)

In [19]:
#not requires but groping number of venues in a neighborhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Armenia Gardens Estates,100,100,100,100,100,100
Audubon Park,100,100,100,100,100,100
Ballast Point,100,100,100,100,100,100
Bayshore Beautiful,100,100,100,100,100,100
Bayshore Gardens,100,100,100,100,100,100
Bayside West,100,100,100,100,100,100
Beach Park,100,100,100,100,100,100
Beach Park Isles,100,100,100,100,100,100
Beasley,100,100,100,100,100,100


In [20]:
#categories of venues in the venue dataframe
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 232 uniques categories.


In [21]:
"Movie Theater" in venues_df['VenueCategory'].unique()

True

checking if we fetched movie theatres

In [22]:
# print out the list of categories
venues_df['VenueCategory'].unique()#[:50]
venues_df[venues_df['VenueCategory']=='Movie Theater']

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
169,Audubon Park,28.077427,-82.416974,Studio Movie Grill Tampa,28.056967,-82.434601,Movie Theater
441,Bayshore Gardens,27.925150,-82.486960,CinéBistro at Hyde Park Village,27.937661,-82.476250,Movie Theater
507,Bayside West,27.935968,-82.526926,AMC West Shore 14,27.946973,-82.526648,Movie Theater
608,Beach Park,27.939480,-82.523240,AMC West Shore 14,27.946973,-82.526648,Movie Theater
707,Beach Park Isles,27.933620,-82.530880,AMC West Shore 14,27.946973,-82.526648,Movie Theater
813,Beasley,28.020627,-82.567491,AMC Veterans 24,28.033060,-82.542809,Movie Theater
980,Bel Mar Shores,27.911670,-82.528940,AMC West Shore 14,27.946973,-82.526648,Movie Theater
1165,Carver City-Lincoln Gardens,27.957410,-82.513830,AMC West Shore 14,27.946973,-82.526648,Movie Theater
1214,Channel District,27.948710,-82.446190,Tampa Theatre,27.950535,-82.458439,Movie Theater
1373,College Hill,27.975970,-82.431690,Tampa Theatre,27.950535,-82.458439,Movie Theater


Analysing each neighborhood

In [23]:
# one hot encoding
Tpo_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tpo_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Tpo_onehot.columns[-1]] + list(Tpo_onehot.columns[:-1])
Tpo_onehot = Tpo_onehot[fixed_columns]

Tpo_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Lounge,Airport Terminal,American Restaurant,Animal Shelter,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Floating Market,Fondue Restaurant,Food & Drink Shop,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Professional & Other Places,Pub,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Armenia Gardens Estates,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Armenia Gardens Estates,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [24]:
Tpo_onehot.shape

(9510, 233)

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
Tpo_grouped = Tpo_onehot.groupby('Neighborhood').mean().reset_index()
Tpo_grouped

,Neighborhood,Adult Boutique,Airport,Airport Lounge,Airport Terminal,American Restaurant,Animal Shelter,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Floating Market,Fondue Restaurant,Food & Drink Shop,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Professional & Other Places,Pub,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Armenia Gardens Estates,0.00,0.00,0.00,0.000000,0.030000,0.00,0.00,0.01,0.00,0.00,0.000000,0.020000,0.00,0.00,0.000000,0.020000,0.00,0.010000,0.00,0.040000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.010000,0.030000,0.00,0.00,0.00,0.010000,0.00,0.00,0.000000,0.010000,0.00,0.00,0.010000,0.00,0.00,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.030000,0.010000,0.000000,0.000000,0.00,0.00,0.000000,0.020000,0.010000,0.01,0.030000,0.010000,0.000000,0.01,0.010000,0.000000,0.00,0.00,0.00,0.01,0.000000,0.01,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.010000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.010000,0.

new size of dataframe

In [26]:
Tpo_grouped.shape

(97, 233)

In [27]:
len(Tpo_grouped[Tpo_grouped['Movie Theater'] > 0])

60

New dataframe from the grouped data containing Movie theaters

In [28]:
Tpo_Theater = Tpo_grouped[["Neighborhood","Movie Theater"]]
Tpo_Theater

,Neighborhood,Movie Theater
0,Armenia Gardens Estates,0.000000
1,Audubon Park,0.010000
2,Ballast Point,0.000000
3,Bayshore Beautiful,0.000000
4,Bayshore Gardens,0.010000
5,Bayside West,0.010000
6,Beach Park,0.010000
7,Beach Park Isles,0.010000
8,Beasley,0.010000
9,Bel Mar Shores,0.010000


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 3 clusters.

In [29]:
# set number of clusters
kclusters = 3

Tpo_grouped_clustering = Tpo_Theater.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=10).fit(Tpo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 0, 0, 0, 0, 0, 0])

Adding cluster labels latitude and longitude to visualise clusters

In [30]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Tpo_merged = Tpo_Theater
Tpo_merged["Cluster Labels"] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#Tpo_merged = Tpo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Tpo_merged.head() # check the last columns!

C:\Users\saisu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Neighborhood,Movie Theater,Cluster Labels
0,Armenia Gardens Estates,0.00,1
1,Audubon Park,0.01,0
2,Ballast Point,0.00,1
3,Bayshore Beautiful,0.00,1
4,Bayshore Gardens,0.01,0


In [31]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Tpo_merged = Tpo_merged.join(Tpo_df.set_index("Neighborhood"), on="Neighborhood")

print(Tpo_merged.shape)
Tpo_merged.head() # check the last columns!


(97, 5)


,Neighborhood,Movie Theater,Cluster Labels,Latitude,Longitude
0,Armenia Gardens Estates,0.00,1,28.018050,-82.481380
1,Audubon Park,0.01,0,28.077427,-82.416974
2,Ballast Point,0.00,1,27.880620,-82.488410
3,Bayshore Beautiful,0.00,1,27.901890,-82.496050
4,Bayshore Gardens,0.01,0,27.925150,-82.486960


In [32]:
# sort the results by Cluster Labels
print(Tpo_merged.shape)
Tpo_merged.sort_values(["Cluster Labels"], inplace=True)
Tpo_merged


(97, 5)


,Neighborhood,Movie Theater,Cluster Labels,Latitude,Longitude
48,North Bon Air,0.010000,0,27.948140,-82.509770
79,Southern Pines,0.010000,0,27.943000,-82.495800
60,Palma Ceia West,0.010000,0,27.927960,-82.509980
78,Southeast Seminole Heights,0.010000,0,27.989080,-82.448050
32,Gray Gables,0.010000,0,27.943857,-82.500010
34,Harvey Heights,0.010000,0,27.958388,-82.447600
76,South Seminole Heights,0.010000,0,27.988760,-82.463530
75,South Nebraska,0.010000,0,27.957340,-82.452520
73,Robles Park,0.010000,0,27.973910,-82.455090
72,Riverside Heights,0.010000,0,27.974010,-82.473740


Finally, let's visualize the resulting clusters

In [33]:
# create map
map_clusters = folium.Map(location=[27.94752,-82.45843], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tpo_merged['Latitude'], Tpo_merged['Longitude'], Tpo_merged['Neighborhood'], Tpo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [34]:
Tpo_merged.loc[Tpo_merged['Cluster Labels'] == 0, Tpo_merged.columns[[1] + list(range(5, Tpo_merged.shape[1]))]]

,Movie Theater
48,0.010000
79,0.010000
60,0.010000
78,0.010000
32,0.010000
34,0.010000
76,0.010000
75,0.010000
73,0.010000
72,0.010000


#### Cluster 2

In [35]:
Tpo_merged.loc[Tpo_merged['Cluster Labels'] == 1, Tpo_merged.columns[[1] + list(range(5, Tpo_merged.shape[1]))]]

,Movie Theater
89,0.0
68,0.0
77,0.0
74,0.0
65,0.0
64,0.0
86,0.0
85,0.0
69,0.0
70,0.0


#### Cluster 3

In [36]:
Tpo_merged.loc[Tpo_merged['Cluster Labels'] == 2, Tpo_merged.columns[[1] + list(range(5, Tpo_merged.shape[1]))]]

,Movie Theater
94,0.02
93,0.02
91,0.02
33,0.02
19,0.02
36,0.02
49,0.02
38,0.02
57,0.02
12,0.02


From the above map we can see movie theatres are more concentrated in the areas of cluster 1 and 3 i.e these neighbourhoods has thetres in the vicinity and the neighbor hoods in the violet  color have less concentration